---

_You are currently looking at **version 1.1** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-text-mining/resources/d9pwm) course resource._

---

# Assignment 1

In this assignment, you'll be working with messy medical data and using regex to extract relevant infromation from the data. 

Each line of the `dates.txt` file corresponds to a medical note. Each note has a date that needs to be extracted, but each date is encoded in one of many formats.

The goal of this assignment is to correctly identify all of the different date variants encoded in this dataset and to properly normalize and sort the dates. 

Here is a list of some of the variants you might encounter in this dataset:
* 04/20/2009; 04/20/09; 4/20/09; 4/3/09
* Mar-20-2009; Mar 20, 2009; March 20, 2009;  Mar. 20, 2009; Mar 20 2009;
* 20 Mar 2009; 20 March 2009; 20 Mar. 2009; 20 March, 2009
* Mar 20th, 2009; Mar 21st, 2009; Mar 22nd, 2009
* Feb 2009; Sep 2009; Oct 2010
* 6/2008; 12/2009
* 2009; 2010

Once you have extracted these date patterns from the text, the next step is to sort them in ascending chronological order accoring to the following rules:
* Assume all dates in xx/xx/xx format are mm/dd/yy
* Assume all dates where year is encoded in only two digits are years from the 1900's (e.g. 1/5/89 is January 5th, 1989)
* If the day is missing (e.g. 9/2009), assume it is the first day of the month (e.g. September 1, 2009).
* If the month is missing (e.g. 2010), assume it is the first of January of that year (e.g. January 1, 2010).
* Watch out for potential typos as this is a raw, real-life derived dataset.

With these rules in mind, find the correct date in each note and return a pandas Series in chronological order of the original Series' indices.

For example if the original series was this:

    0    1999
    1    2010
    2    1978
    3    2015
    4    1985

Your function should return this:

    0    2
    1    4
    2    0
    3    1
    4    3

Your score will be calculated using [Kendall's tau](https://en.wikipedia.org/wiki/Kendall_rank_correlation_coefficient), a correlation measure for ordinal data.

*This function should return a Series of length 500 and dtype int.*

In [40]:
n = 25
import pandas as pd

doc = []
with open('dates.txt') as file:
    for line in file:
        doc.append(line)

df = pd.Series(doc)
df.head(n)

0          03/25/93 Total time of visit (in minutes):\n
1                        6/18/85 Primary Care Doctor:\n
2     sshe plans to move as of 7/8/71 In-Home Servic...
3                 7 on 9/27/75 Audit C Score Current:\n
4     2/6/96 sleep studyPain Treatment Pain Level (N...
5                     .Per 7/06/79 Movement D/O note:\n
6     4, 5/18/78 Patient's thoughts about current su...
7     10/24/89 CPT Code: 90801 - Psychiatric Diagnos...
8                          3/7/86 SOS-10 Total Score:\n
9              (4/10/71)Score-1Audit C Score Current:\n
10    (5/11/85) Crt-1.96, BUN-26; AST/ALT-16/22; WBC...
11                        4/09/75 SOS-10 Total Score:\n
12    8/01/98 Communication with referring physician...
13    1/26/72 Communication with referring physician...
14    5/24/1990 CPT Code: 90792: With medical servic...
15    1/25/2011 CPT Code: 90792: With medical servic...
16          4/12/82 Total time of visit (in minutes):\n
17         1; 10/13/1976 Audit C Score, Highest/

In [145]:
import re
import datetime


months = ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']

def normalize_date(d, m, y):
    dd = mm = yyyy = ''
    dd = '0' + d if len(d) == 1 else d
    mm = '0' + m if len(m) == 1 else m
    yyyy = '19' + y if len(y) == 2 else y
    return '{0}-{1}-{2}'.format(yyyy, mm, dd)
    
def date_sorter():
    # Your code here
    regex_list = [
        r'(\d{1,2})[/-](\d{1,2})[/-](\d{2,4})', # 04/20/2009
        r'(\d{1,2})[ -](Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)[\w\.]*[ ,-]+(\d{2,4})', # 20 Mar 2009
        r'(Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)[\w\.]*[ ,-]+((?:19|20)(?:\d{2}))', # Mar 2009
        r'(Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)[\w\.]*[ -](\d{1,2})\w*[ ,-]+((?:19|20)(?:\d{2}))', # Mar 20, 2009
        r'(\d{1,2})/((?:19|20)(?:\d{2}))', # 6/2009
        r'((?:19|20)(?:\d{2}))' # 2009
    ]
    
    ll = pd.Series([])
      
    for i, line in df.items():
        for ri, rx in enumerate(regex_list):
            se = re.search(rx, line)
            if (se):
                ans = None
                d = se.groups()
                if (ri == 0):   
                    ans = normalize_date(d[1], d[0], d[2])
                elif (ri == 1): 
                    ans = normalize_date(d[0], str(months.index(d[1]) + 1), d[2])
                elif (ri == 2):
                    ans = normalize_date('01', str(months.index(d[0]) + 1), d[1])
                elif (ri == 3):
                    ans = normalize_date(d[1], str(months.index(d[0]) + 1), d[2])
                elif (ri == 4):
                    ans = normalize_date('01', d[0], d[1])
                elif (ri == 5):
                    ans = normalize_date('01', '01', d[0])
                ll[i] = ans
                break
                
    return pd.Series(data=ll.index.values, index=ll.values).sort_index() # Your answer here

1971-04-10      9
1971-05-18     84
1971-07-08      2
1971-07-11     53
1971-09-12     28
1972-01-01    474
1972-01-13    153
1972-01-26     13
1972-05-06    129
1972-05-13     98
1972-06-10    111
1972-06-15    225
1972-07-20     31
1972-10-04    171
1972-11-30    191
1973-01-01    486
1973-02-01    335
1973-02-01    415
1973-02-14     36
1973-03-01    405
1973-03-01    323
1973-04-01    422
1973-06-01    375
1973-07-01    380
1973-10-01    345
1973-12-01     57
1974-01-01    481
1974-02-01    436
1974-02-24    104
1974-03-01    299
             ... 
2012-06-25    220
2012-09-01    208
2012-09-01    243
2012-10-21    139
2012-11-01    320
2012-12-01    383
2013-01-01    244
2013-01-01    286
2013-01-01    480
2013-04-01    431
2013-09-01    279
2013-10-11    198
2014-01-01    381
2014-01-01    463
2014-07-01    366
2014-10-01    439
2014-10-01    255
2014-12-01    401
2015-01-01    475
2015-09-01    257
2015-09-28    152
2015-10-01    235
2016-01-01    464
2016-02-01    253
2016-05-01